# PURPOSE OF THIS NOTEBOOK
This is the base code to write the functions to extract the raw text from resumes and append to our dataframe. 

TODO: Take the functions into a .py folder and use it as a script 

In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from tokenizers import ByteLevelBPETokenizer
from transformers import BertTokenizer, BertModel
from pathlib import Path
from sklearn.model_selection import train_test_split
from torch import cuda
import torch

/home/omah/Documents/GitHub/ResumeSkillExtraction/resumeenv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Model goal

My goal is to load in a model from Hugging Face and train with the ONET data, This will allow me to utilize the power of the models on hugging face. I would need to start training this model on my data to see how it would preform.

In [2]:
# Import the train/test Data. 
test_df = pd.read_csv("../Data/TestingData.csv")
train_df = pd.read_csv("../Data/Training_Data.csv")

In [3]:
# Check that the test data incoming is correct
test_df.head()

,Reported_Jobs,Label
0,Chief Diversity Officer (CDO),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Operations Vice President (Operations VP),"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Agricultural Services Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Bureau Chief,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Business Development Executive,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [4]:
# Check that the training data incoming is correct 
train_df.head()

,Reported_Jobs,Label
0,Road Commissioner,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,Tax Commissioner,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,Deputy Insurance Commissioner,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,School Commissioner,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,Aeronautics Commission Director,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


## The goal of the tokenizer

The tokenizer will be tokenizing the job titles and the reported job titles, I don't know if I should do these seperately or together. In theory, I should have these pairings be tokenized together. **I SHOULD READ INTO THE TOKENIZER TO UNDERSTAND HOW I SHOULD APPROACH THIS** 

From looking at the reference code, I've learned that we need to follow these steps: 
1. Start with a train test split. 30% of the of the test data, I will do the split based on the 30% of each reported job title/ONET pairing.
    - Given the refernece notebook uses a dictionary as the input data and we are working with a dataframe instead, some major changes will be needed to be made in order for this to work. I don't think this would be difficult at all. Just need to translate the dictionary work to the dataframe. **I also need to confirm if the model input requires a list, dict, or dataframe object.**
2. Run the tokenizer on the training set 
3. Set up the model training and evaluation metrics.


In [5]:
# This is text extraction, Code will be deprecated for now until I find use for it later. 
text_data = []
file_count=0

for sample in tqdm(train_df['Reported_Jobs']):
    text_data.append(sample)
    if len(text_data) == 10_000:
        # once we git the 100K mark, save to file
        with open(f'../Data/text_{file_count}.txt', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(text_data))
        text_data = []
        file_count += 1

100%|██████████| 31166/31166 [00:00<00:00, 4777591.41it/s]


In [6]:
# Created the data for the tokenizer, now to 

paths = [str(x) for x in Path('../Data/').glob('**/*.txt')]

tokenizer = ByteLevelBPETokenizer()

In [7]:
# Train the tokenizer

tokenizer.train(files=paths[:5], vocab_size=30_522, min_frequency=2, special_tokens=['<s>', '<pad>', '</s>', '<unk>', '<mask>'])

# Save the tokenizer 
tokenizer.save_model(directory="../Data")

['../Data/vocab.json', '../Data/merges.txt']

In [8]:
from transformers import RobertaTokenizer

# initialize the tokenizer
tokenizer = RobertaTokenizer.from_pretrained('../Data', max_len=512)

## The validation data. 

Since this dataset that I'm using to test does not have ONET codes attached to them, I would have to source another dataset that does. Thankfully ONET does have such data that I can use for training. For each ONET code there's common job names appended to them, I could use that data with the pretained model. That being said this creates some challenges. 
1. For obsure and odd job names that are not listed in the common job names will almost guarentee bad performance. Given that the goal is to prove that we can get generally good performance with publicly available data, this is something I'm willing to sacrifice if it means that it will work correctly at least 80% of the time. 

I have sourced a table with common names linked with each onet code. I can use this data on the pretrained model to see how it classifies each of the common names linked with ONET codes. 

## Next steps

After loading in the pretrained model, I want to load the data into this model and test the performance while holding back around 30% of the data for testing. After I complete the testing phase of the model performance, I will start the next step and pull, or train a model or a script to pull the job titles from a resume and apply into the model to classify into an appropriote ONET code. 

In [9]:
def masking(tensor):
    rand = torch.rand(tensor.shape)
    mask_arr = (rand < 0.15) * (tensor > 2)
    for i in range(tensor.shape[0]):
        selection = torch.flatten(mask_arr[i].nonzero()).tolist()
        tensor[i, selection] = 4
    return tensor

In [10]:
from tqdm.auto import tqdm 

input_ids = []
mask = []
labels = []

for path in tqdm(paths):
    with open(path, 'r', encoding='utf-8') as f:
        lines = f.read().split('\n')
    sample = tokenizer(lines, max_length=512, padding='max_length', truncation=True, return_tensors='pt')
    labels.append(sample.input_ids)
    mask.append(sample.attention_mask)
    input_ids.append(masking(sample.input_ids.detach().clone()))

100%|██████████| 4/4 [00:04<00:00,  1.24s/it]


In [11]:
input_ids = torch.cat(input_ids)
labels = torch.cat(labels)
mask = torch.cat(mask)

In [12]:
encodings = {
    'input_ids' : input_ids,
    'attention_mask' : mask,
    'labels': labels
}

In [13]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    
    def __len__(self):
        return self.encodings['input_ids'].shape[0]
    
    def __getitem__(self, i):
        return {key: tensor[i] for key, tensor in self.encodings.items()}

In [14]:
dataset = Dataset(encodings)

In [15]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

## Train the Model



In [16]:
from transformers import RobertaConfig

In [17]:
config = RobertaConfig(
    vocab_size=tokenizer.vocab_size,
    max_position_embeddings=514,
    hidden_size=768,
    num_attention_heads=12,
    num_hidden_layers=3,
    type_vocab_size=1
)

In [18]:
from transformers import RobertaForMaskedLM

In [19]:
model = RobertaForMaskedLM(config)

In [20]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [21]:
model.to(device)

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(13270, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-2): 3 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): La

In [22]:
from transformers import AdamW

In [23]:
optim = AdamW(model.parameters(), lr=1e-5)

/home/omah/Documents/GitHub/ResumeSkillExtraction/resumeenv/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [26]:
epochs = 2

for epoch in range(epochs):
    loop = tqdm(dataloader, leave=True)
    for batch in loop:
        optim.zero_grad()

        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)

        # extract loss
        loss = outputs.loss

        loss.backward()

        optim.step()

        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())


Epoch 0:   3%|▎         | 91/2689 [00:46<21:43,  1.99it/s, loss=8.31]

In [ ]:
model.save_pretrained('../Data')

## File mask testing

In [25]:
from transformers import pipeline 

fill = pipeline('fill-mask', model='../Data', tokenizer='../Data')

OSError: ../Data does not appear to have a file named config.json. Checkout 'https://huggingface.co/../Data/None' for available files.